# Rolling Ratings for 10 game sample

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [ ]:
year = 2023
df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{year}.parquet")
df1 = df1.rename(
        columns={
            "offensiveRating": "ORtg",
            "defensiveRating": "DRtg",
            "netRating": "NRtg",
            "possessions": "poss",
        }
)
cols = ["gameId", "teamId", "ORtg", "DRtg", "NRtg", "poss"]
df1 = df1[cols]
df2 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{year}.parquet")
df2 = df2.rename(
    columns={
        "GAME_ID": "gameId",
        "TEAM_ID": "teamId",
        "TEAM_ABBREVIATION": "teamTricode",
        "GAME_DATE": "gameDate",
        "TEAM_NAME": "teamName",
    }
)
cols2 = ["gameId", "teamId", "teamTricode", "gameDate"]
df2 = df2[cols2]
df2["gameId"] = df2["gameId"].astype(int)
df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
df3 = df3.rename(columns={"gameDate": "Date"})
df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")
df4 = df3.sort_values(by=["teamTricode", "Date"]).reset_index(drop=True)
teams = df4["teamTricode"].unique()
df4g = df4.groupby("teamTricode")
team = teams[0]
df6 = []
for team in teams:
    df5 = df4g.get_group(team)
    df5[["rORtg","rDRtg","rNRtg"]] = df5[["ORtg","DRtg","NRtg"]].rolling(10,min_periods=1).mean()
    df5[["rORtg","rDRtg","rNRtg"]] = df5[["rORtg","rDRtg","rNRtg"]].round(1)
    df6.append(df5)
df6 = pd.concat(df6)
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam": "Team"})
df7 = pd.merge(df6, df_teams)
data = df7
data["teamDup"] = data["teamTricode"]
data1 = data.copy().drop(columns=["teamTricode"])

In [ ]:
team = "TOR"
data2 = data[data["teamTricode"] == team].reset_index(drop=True)

var = "NRtg"
title = data2["Team"][0] + " Net Rating 2023-24"
subtitle = "League Avg is red | Trend line estimated using local regression"

p = (
    ggplot(data2)
    # + geom_smooth(
    #     data=data1,
    #     mapping=aes(x="Date", y=var, group="teamDup"),
    #     method="loess",
    #     color="lightgrey",
    #     se=False,
    #     size=0.5,
    # )
    + geom_smooth(
        aes(x="Date", y=var, group="teamTricode"),
        # method="loess",
        color="Black",
        se=False,
        span=0.5
    )
    + geom_point(
        aes(x="Date", y=var, group="teamTricode"),
        color="Black",
    )
    # + geom_smooth(data= data_avg,mapping=aes(x="Date", y=var,group = "teamDup"),method="loess",color="red", se=False, size=0.5)
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    # + ylim(-10, 10)
    + scale_x_date(date_labels="%b-%d", date_breaks="2 week")
    # + ylim(100, 130)
    # + coord_cartesian(ylim=[-10,10])
    + labs(
        title=title,
        subtitle=subtitle,
        caption="@SravanNBA | source:nba.com/stats",
    )
    + theme_idv
)
p.save(fig_DIR + "Rolling_" + var + "_" + team + ".png", width=6, height=5, dpi=300)
p.draw()